In [2]:
import tensorflow as tf

In [3]:
import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

/Users/livid/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TensorFlow version: 2.9.2
Num GPUs Available:  1
Num CPUs Available:  1


In [4]:
!pip install tf_slim
!pip install pycocotools

In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2

In [7]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import pathlib
! git clone --depth 1 https://github.com/tensorflow/models

2022-07-11 20:16:34.658 xcodebuild[62806:2898407] Requested but did not find extension point with identifier Xcode.IDEKit.ExtensionSentinelHostApplications for extension Xcode.DebuggerFoundation.AppExtensionHosts.watchOS of plug-in com.apple.dt.IDEWatchSupportCore
2022-07-11 20:16:34.658 xcodebuild[62806:2898407] Requested but did not find extension point with identifier Xcode.IDEKit.ExtensionPointIdentifierToBundleIdentifier for extension Xcode.DebuggerFoundation.AppExtensionToBundleIdentifierMap.watchOS of plug-in com.apple.dt.IDEWatchSupportCore
^C


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

Processing /Users/livid/ML/ssd_tf2/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached lxml-4.9.1.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Cython-0.29.30-py2.py3-none-any.whl (985 kB)
  Using cached contextlib2-21.6.0-py2.py3-none-any.whl (13 kB)
  Using cached lvis-0.5.3-py3-none-any.whl (14 kB)
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1653598 sha256=37a873eb12164f0293383cf9b1de942dc48b33549d93a98af57c1ad034e63688
  Stored in directory: /private/var/folders/0r/2q6y8l_55lbd8rrwd9zxvs500000gn/T/pip-ephem-wheel-cache-cc7wzo_7/wheels/88/5e/29/bad53c2d291c6ae85b93d7a47b63ea3fbb03b4d39fb7e79ab5


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [219 lines of output]
      Building lxml version 4.9.1.
      Building without Cython.
      Building against libxml2 2.9.4 and libxslt 1.1.29
      Building against libxml2/libxslt in the following directory: /Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/lib
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.0-arm64-cpython-39
      creating build/lib.macosx-11.0-arm64-cpython-39/lxml
      copying src/lxml/_elementpath.py -> build/lib.macosx-11.0-arm64-cpython-39/lxml
      copying src/lxml/sax.py -> build/lib.macosx-11.0-arm64-cpython-39/lxml
      copying src/lxml/pyclasslookup.py -> build/lib.macosx-11.0-arm64-cpython-39/lxml
      copying src/lxml/__init__.py -> build/lib.macosx-11.0-arm64-cpython-39/lxml
      copying src/lxml/builder.py -> build/lib.macosx-11.0-arm64-cpython

  Running setup.py clean for lxml
Successfully built object-detection
Failed to build lxml
  Running setup.py install for lxml: started
  Running setup.py install for lxml: finished with status 'done'


  DEPRECATION: lxml was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https://github.com/pypa/pip/issues/8368


In [9]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [10]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/Users/livid/miniforge3/lib/python3.9/site-packages/google/protobuf/internal/__init__.py)

In [11]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('Car')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpeg")))


In [12]:
tflite_path = pathlib.Path.cwd()/"ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8"/"model.tflite"
tflite_quan_path = "ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/model_quan.tflite"
tflite_float_fallback_path = "ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/model_quan_float_fallback.tflite"
tflite_full_int_path = "ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/model_quan_full_int.tflite"

interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_quan_path))
interpreter_quant.allocate_tensors()

interpreter_float_fallback = tf.lite.Interpreter(model_path=str(tflite_float_fallback_path))
interpreter_float_fallback.allocate_tensors()

interpreter_full_int = tf.lite.Interpreter(model_path=str(tflite_full_int_path))
interpreter_full_int.allocate_tensors()

In [13]:
def test_lite(img_path, interpreter):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = img_rgb.astype(np.float32)/255
    img_rgb = cv2.resize(img_rgb, (640, 640), cv2.INTER_AREA)
    img_rgb = img_rgb.reshape([1, 640, 640, 3])

    interpreter.set_tensor(input_details[0]['index'], img_rgb)
    interpreter.invoke()
    output_dict = {
                    'detection_boxes' : interpreter.get_tensor(output_details[0]['index'])[0],
                    'detection_classes' : interpreter.get_tensor(output_details[1]['index'])[0],
                    'detection_scores' : interpreter.get_tensor(output_details[2]['index'])[0],
                    'num_detections' : interpreter.get_tensor(output_details[3]['index'])[0]
                    }

In [15]:
%%time

for i in range(1):
    for image_path in TEST_IMAGE_PATHS[:100]:
        print(image_path)
        test_lite(str(image_path), interpreter)

Car/03099.jpeg
Car/03100.jpeg
Car/03101.jpeg
Car/03103.jpeg
Car/03105.jpeg
Car/03106.jpeg
Car/03111.jpeg
Car/03113.jpeg
Car/03114.jpeg
Car/03115.jpeg
Car/03116.jpeg
Car/03117.jpeg
Car/03120.jpeg
Car/03121.jpeg
Car/03122.jpeg
Car/03123.jpeg
Car/03124.jpeg
Car/03125.jpeg
Car/03127.jpeg
Car/03128.jpeg
Car/03130.jpeg
Car/03131.jpeg
Car/03132.jpeg
Car/03133.jpeg
Car/03134.jpeg
Car/03135.jpeg
Car/03136.jpeg
Car/03137.jpeg
Car/03139.jpeg
Car/03140.jpeg
Car/03142.jpeg
Car/03145.jpeg
Car/03151.jpeg
Car/03152.jpeg
Car/03154.jpeg
Car/03156.jpeg
Car/03157.jpeg
Car/03158.jpeg
Car/03159.jpeg
Car/03162.jpeg
Car/03163.jpeg
Car/03164.jpeg
Car/03165.jpeg
Car/03169.jpeg
Car/03170.jpeg
Car/03172.jpeg
Car/03173.jpeg
Car/03175.jpeg
Car/03177.jpeg
Car/03178.jpeg
Car/03179.jpeg
Car/03180.jpeg
Car/03183.jpeg
Car/03184.jpeg
Car/03185.jpeg
Car/03187.jpeg
Car/03188.jpeg
Car/03195.jpeg
Car/03196.jpeg
Car/03200.jpeg
Car/03201.jpeg
Car/03202.jpeg
Car/03203.jpeg
Car/03204.jpeg
Car/03206.jpeg
Car/03207.jpeg
Car/03208.

In [16]:
%%time

for i in range(1):
    for image_path in TEST_IMAGE_PATHS[:100]:
        print(image_path)
        test_lite(str(image_path), interpreter_quant)

Car/03099.jpeg
Car/03100.jpeg
Car/03101.jpeg
Car/03103.jpeg
Car/03105.jpeg
Car/03106.jpeg
Car/03111.jpeg
Car/03113.jpeg
Car/03114.jpeg
Car/03115.jpeg
Car/03116.jpeg
Car/03117.jpeg
Car/03120.jpeg
Car/03121.jpeg
Car/03122.jpeg
Car/03123.jpeg
Car/03124.jpeg
Car/03125.jpeg
Car/03127.jpeg
Car/03128.jpeg
Car/03130.jpeg
Car/03131.jpeg
Car/03132.jpeg
Car/03133.jpeg
Car/03134.jpeg
Car/03135.jpeg
Car/03136.jpeg
Car/03137.jpeg
Car/03139.jpeg
Car/03140.jpeg
Car/03142.jpeg
Car/03145.jpeg
Car/03151.jpeg
Car/03152.jpeg
Car/03154.jpeg
Car/03156.jpeg
Car/03157.jpeg
Car/03158.jpeg
Car/03159.jpeg
Car/03162.jpeg
Car/03163.jpeg
Car/03164.jpeg
Car/03165.jpeg
Car/03169.jpeg
Car/03170.jpeg
Car/03172.jpeg
Car/03173.jpeg
Car/03175.jpeg
Car/03177.jpeg
Car/03178.jpeg
Car/03179.jpeg
Car/03180.jpeg
Car/03183.jpeg
Car/03184.jpeg
Car/03185.jpeg
Car/03187.jpeg
Car/03188.jpeg
Car/03195.jpeg
Car/03196.jpeg
Car/03200.jpeg
Car/03201.jpeg
Car/03202.jpeg
Car/03203.jpeg
Car/03204.jpeg
Car/03206.jpeg
Car/03207.jpeg
Car/03208.

In [17]:
%%time

for i in range(1):
    for image_path in TEST_IMAGE_PATHS[:100]:
        print(image_path)
        test_lite(str(image_path), interpreter_float_fallback)

Car/03099.jpeg
Car/03100.jpeg
Car/03101.jpeg
Car/03103.jpeg
Car/03105.jpeg
Car/03106.jpeg
Car/03111.jpeg
Car/03113.jpeg
Car/03114.jpeg
Car/03115.jpeg
Car/03116.jpeg
Car/03117.jpeg
Car/03120.jpeg
Car/03121.jpeg
Car/03122.jpeg
Car/03123.jpeg
Car/03124.jpeg
Car/03125.jpeg
Car/03127.jpeg
Car/03128.jpeg
Car/03130.jpeg
Car/03131.jpeg
Car/03132.jpeg
Car/03133.jpeg
Car/03134.jpeg
Car/03135.jpeg
Car/03136.jpeg
Car/03137.jpeg
Car/03139.jpeg
Car/03140.jpeg
Car/03142.jpeg
Car/03145.jpeg
Car/03151.jpeg
Car/03152.jpeg
Car/03154.jpeg
Car/03156.jpeg
Car/03157.jpeg
Car/03158.jpeg
Car/03159.jpeg
Car/03162.jpeg
Car/03163.jpeg
Car/03164.jpeg
Car/03165.jpeg
Car/03169.jpeg
Car/03170.jpeg
Car/03172.jpeg
Car/03173.jpeg
Car/03175.jpeg
Car/03177.jpeg
Car/03178.jpeg
Car/03179.jpeg
Car/03180.jpeg
Car/03183.jpeg
Car/03184.jpeg
Car/03185.jpeg
Car/03187.jpeg
Car/03188.jpeg
Car/03195.jpeg
Car/03196.jpeg
Car/03200.jpeg
Car/03201.jpeg
Car/03202.jpeg
Car/03203.jpeg
Car/03204.jpeg
Car/03206.jpeg
Car/03207.jpeg
Car/03208.

In [19]:
def test_lite_int(img_path, interpreter):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (640, 640), cv2.INTER_AREA)
    img_rgb = img_rgb.reshape([1, 640, 640, 3])

    interpreter.set_tensor(input_details[0]['index'], img_rgb)
    interpreter.invoke()
    output_dict = {
                    'detection_boxes' : interpreter.get_tensor(output_details[0]['index'])[0],
                    'detection_classes' : interpreter.get_tensor(output_details[1]['index'])[0],
                    'detection_scores' : interpreter.get_tensor(output_details[2]['index'])[0],
                    'num_detections' : interpreter.get_tensor(output_details[3]['index'])[0]
                    }

In [20]:
%%time

for i in range(1):
    for image_path in TEST_IMAGE_PATHS[:100]:
        print(image_path)
        test_lite_int(str(image_path), interpreter_full_int)

Car/03099.jpeg
Car/03100.jpeg
Car/03101.jpeg
Car/03103.jpeg
Car/03105.jpeg
Car/03106.jpeg
Car/03111.jpeg
Car/03113.jpeg
Car/03114.jpeg
Car/03115.jpeg
Car/03116.jpeg
Car/03117.jpeg
Car/03120.jpeg
Car/03121.jpeg
Car/03122.jpeg
Car/03123.jpeg
Car/03124.jpeg
Car/03125.jpeg
Car/03127.jpeg
Car/03128.jpeg
Car/03130.jpeg
Car/03131.jpeg
Car/03132.jpeg
Car/03133.jpeg
Car/03134.jpeg
Car/03135.jpeg
Car/03136.jpeg
Car/03137.jpeg
Car/03139.jpeg
Car/03140.jpeg
Car/03142.jpeg
Car/03145.jpeg
Car/03151.jpeg
Car/03152.jpeg
Car/03154.jpeg
Car/03156.jpeg
Car/03157.jpeg
Car/03158.jpeg
Car/03159.jpeg
Car/03162.jpeg
Car/03163.jpeg
Car/03164.jpeg
Car/03165.jpeg
Car/03169.jpeg
Car/03170.jpeg
Car/03172.jpeg
Car/03173.jpeg
Car/03175.jpeg
Car/03177.jpeg
Car/03178.jpeg
Car/03179.jpeg
Car/03180.jpeg
Car/03183.jpeg
Car/03184.jpeg
Car/03185.jpeg
Car/03187.jpeg
Car/03188.jpeg
Car/03195.jpeg
Car/03196.jpeg
Car/03200.jpeg
Car/03201.jpeg
Car/03202.jpeg
Car/03203.jpeg
Car/03204.jpeg
Car/03206.jpeg
Car/03207.jpeg
Car/03208.